In [1]:
!pip install procyclingstats

In [2]:
import pandas as pd
world_tour_races = pd.read_csv('../data/pcs_worldtour_races.csv')
world_tour_races = world_tour_races.query('year >= 2016')
world_tour_races

,race_name,year,stage_race,race_url
83,tour-down-under,2016,True,race/tour-down-under/2016
84,paris-nice,2016,True,race/paris-nice/2016
85,tirreno-adriatico,2016,True,race/tirreno-adriatico/2016
86,milano-sanremo,2016,False,race/milano-sanremo/2016
87,volta-a-catalunya,2016,True,race/volta-a-catalunya/2016
...,...,...,...,...
322,benelux-tour,2022,True,race/benelux-tour/2022
323,gp-quebec,2022,False,race/gp-quebec/2022
324,gp-montreal,2022,False,race/gp-montreal/2022
325,il-lombardia,2022,False,race/il-lombardia/2022


In [3]:
from procyclingstats import Stage, Race
from tqdm import tqdm

riders = pd.DataFrame(columns = ['rider_url', 'rider_name', 'nationality'])
teams = pd.DataFrame(columns = ['team_url', 'team_name'])
results = pd.DataFrame(columns = ['rider_url', 'team_url', 'rank', 'status', 'age', 'time', 'pcs_points', 'uci_points'])

def add_results_from_stage(race_url):
    global riders, teams, results
    
    try:
        stage = Stage(race_url)
    except ValueError:
        print(f"Unable to add results from {race_url} - bad URL")
        return
    
    try:
        parsed = stage.parse()
    except IndexError:
        print(f"Unable to add results from {race_url} - bad HTML (possibly TTT or cancelled)")
        return
    except:
        print(f"Unable to add results from {race_url} - bad HTML")
        return
    
    full_df = pd.DataFrame(parsed['results'])
    
    # Clean links   
    full_df['rider_url'] = full_df['rider_url'].str.removeprefix('rider/')
    full_df['team_url'] = full_df['team_url'].str.removeprefix('team/')
    full_df['race_url'] = race_url
    
    # Add other race information
    full_df['profile_icon'] = parsed['profile_icon']
    full_df['profile_score'] = parsed['profile_score']
    
    rider = full_df[['rider_url', 'rider_name', 'nationality']]
    riders = pd.concat([riders, rider]).drop_duplicates()
    team = full_df[['team_url', 'team_name']]
    teams = pd.concat([teams, team]).drop_duplicates()
    result = full_df[[
        'race_url', 'rider_url', 'team_url', 'rank', 'status', 'age', 'time',
        'pcs_points', 'uci_points', 'profile_icon', 'profile_score'
    ]]
    results = pd.concat([results, result])
    
def get_stage_urls(race_url):
    stage = Race(f"{race_url}/overview")
    stages = stage.parse()['stages']
    return [stage['stage_url'] for stage in stages]

for i, race_url in tqdm(world_tour_races.query('not stage_race').race_url.items()):
    add_results_from_stage(race_url)
    
for i, race_url in tqdm(world_tour_races.query('stage_race').race_url.items()):
    stage_urls = get_stage_urls(race_url)
    for stage in stage_urls:
        add_results_from_stage(stage)
        
riders.to_csv('../data/pcs_worldtour_riders.csv')
teams.to_csv('../data/pcs_worldtour_teams.csv')
results.to_csv('../data/pcs_worldtour_results.csv')

2it [00:00,  1.98it/s]

Unable to add results from race/e3-harelbeke/2016 - bad HTML


35it [00:20,  1.04s/it]

Unable to add results from race/great-ocean-race/2018 - bad HTML


47it [00:27,  2.10it/s]

...

Unable to add results from race/benelux-tour/2022/stage-1 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/benelux-tour/2022/stage-2 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/benelux-tour/2022/stage-3 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/benelux-tour/2022/stage-4 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/benelux-tour/2022/stage-5 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/benelux-tour/2022/stage-6 - bad HTML (possibly TTT or cancelled)


107it [11:45,  9.19s/it]

Unable to add results from race/benelux-tour/2022/stage-7 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/tour-of-guangxi/2022/stage-1 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/tour-of-guangxi/2022/stage-2 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/tour-of-guangxi/2022/stage-3 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/tour-of-guangxi/2022/stage-4 - bad HTML (possibly TTT or cancelled)
Unable to add results from race/tour-of-guangxi/2022/stage-5 - bad HTML (possibly TTT or cancelled)


108it [11:48,  6.56s/it]

Unable to add results from race/tour-of-guangxi/2022/stage-6 - bad HTML (possibly TTT or cancelled)
